# USG grants crawl
## Agency-specific replication from  Lee & Chung (2022)

### Previously

In the previous chapter we replicated some of our previous analysis, which was done with a list of open-science terms of our own creation, with an emperically derived list of terms derived from [Lee & Chung (2022)](https://doi.org/10.47989/irpaper949).  While the reults look compelling, and at least appear to plausible relatie to our intuitions, we should always be skeptical of our results until we do some thorough sanity checks.  

In this chapter we'll implement some sanity checks.  These will include confirming that the extreme ends of the term-matching distributions (e.g. many term matches vs no term matches) do indeed reflect the sorts of grants we would expect.  We'll also check to make sure that our quantative analyses of these descriptions actually return the sorts of results we would expect.

### Loading the database once more

Let's begin by loading up the database provided by the website, which is stored in an xml format.

In [1]:
# import our helper functions
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))
sys.path
import grantsGov_utilities as grantsGov_utilities

# local data storage directory
localDataDir='inputData'

grantsDF=grantsGov_utilities.detectLocalGrantData(localPath='../'+localDataDir,forceDownload=True)
grantsDF

,OpportunityID,OpportunityTitle,OpportunityNumber,AgencyCode,AgencyName,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,DOS-SA,Bureau of South and Central Asian Affairs,600000,400000,600000,1,The Office of Press and Public Diplomacy of th...
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,DOI-FWS,Fish and Wildlife Service,0,0,0,NaN,Funds under this award are to be used for the ...
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,DOI-USGS1,Geological Survey,0,0,31900,1,The USGS Southeast Ecological Science Center s...
3,131094,Plant Feedstock Genomics for Bioenergy: A Joi...,DE-FOA-0000598,PAMS-SC,Office of Science,500000,200000,6000000,10,The U.S. Department of Energy&apos;s Office of...
4,131095,Management of HIV-Related Lung Disease and Car...,RFA-HL-12-034,HHS-NIH11,National Institutes of Health,400000,NaN,2000000,NaN,This FOA invites clinical trials planning gran...
...,...,...,...,...,...,...,...,...,...,...
70325,262109,2014/2015 Social Responsibility through Englis...,RELO-BP-MOB,DOS-HUN,U.S. Mission to Hungary,152418,152418,152418,1,"In close consultation with RELO Budapest, the ..."
70326,262108,"Notice of Intent to Award - Fort McHenry, Balt...",NPS-14-NERO-0124,DOI-NPS,National Park Service,65000,65000,65000,1,United States Department of the Interior Natio...
70327,262112,Fish and Wildlife Coordination Act,R14AS00070,DOI-BOR,Bureau of Reclamation,525000,525000,525000,1,"To provide financial assistance, through grant..."
70328,131053,USAID/Uganda Literacy and Health Education Pro...,RFA-617-12-000001,USAID-UGA,Uganda USAID-Kampala,57000000,0,57000000,1,Literacy Program is a 5-year program to improv...


## Cleaning
As before, we need to do a bit of cleaning, so lets do a more comprehensive version of that here.

Note:  This may take a moment

In [2]:
grantsDF=grantsGov_utilities.prepareGrantsDF(grantsDF, repair=True)
grantsDF

OpportunityID                    int64
OpportunityTitle                object
OpportunityNumber               object
AgencyCode                      object
AgencyName                      object
AwardCeiling                     int64
AwardFloor                       int64
EstimatedTotalProgramFunding     int64
ExpectedNumberOfAwards           int64
Description                     object
dtype: object
62144 grant agency name or code value records altered


C:\Users\dbullock\Documents\code\gitDir\USG_grants_crawl\src\grantsGov_utilities.py:216: UserWarning: NOTE: this function CHANGES the values / content of the grantsDF from the information contained on grants.gov, including but not limited to adding data columns, replacing null/empty values, and/or inferring missing values.
  warn('NOTE: this function CHANGES the values / content of the grantsDF from the information contained on grants.gov, including but not limited to adding data columns, replacing null/empty values, and/or inferring missing values.')


28027 grant funding value records repaired


,OpportunityID,OpportunityTitle,OpportunityNumber,AgencyCode,AgencySubCode,AgencyName,AwardCeiling,AwardFloor,EstimatedTotalProgramFunding,ExpectedNumberOfAwards,Description
0,262148,Establishment of the Edmund S. Muskie Graduate...,SCAPPD-14-AW-161-SCA-08152014,DOS,SA,Bureau of South and Central Asian Affairs,600000,400000,600000,1,The Office of Press and Public Diplomacy of th...
1,262149,Eradication of Yellow Crazy Ants on Johnston A...,F14AS00402,DOI,FWS,Fish and Wildlife Service,0,0,0,0,Funds under this award are to be used for the ...
2,131073,"Cooperative Ecosystem Studies Unit, Piedmont S...",G12AS20003,DOI,USGS1,Geological Survey,0,0,31900,1,The USGS Southeast Ecological Science Center s...
3,131094,Plant Feedstock Genomics for Bioenergy: A Joi...,DE-FOA-0000598,PAMS,SC,Office of Science,500000,200000,6000000,10,The U.S. Department of Energy&apos;s Office of...
4,131095,Management of HIV-Related Lung Disease and Car...,RFA-HL-12-034,HHS,NIH11,National Institutes of Health,400000,0,2000000,0,This FOA invites clinical trials planning gran...
...,...,...,...,...,...,...,...,...,...,...,...
70325,262109,2014/2015 Social Responsibility through Englis...,RELO-BP-MOB,DOS,HUN,U.S. Mission to Hungary,152418,152418,152418,1,"In close consultation with RELO Budapest, the ..."
70326,262108,"Notice of Intent to Award - Fort McHenry, Balt...",NPS-14-NERO-0124,DOI,NPS,National Park Service,65000,65000,65000,1,United States Department of the Interior Natio...
70327,262112,Fish and Wildlife Coordination Act,R14AS00070,DOI,BOR,Bureau of Reclamation,525000,525000,525000,1,"To provide financial assistance, through grant..."
70328,131053,USAID/Uganda Literacy and Health Education Pro...,RFA-617-12-000001,USAID,UGA,Uganda USAID-Kampala,57000000,0,57000000,1,Literacy Program is a 5-year program to improv...


In [15]:
# from https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
import gensim
from gensim.utils import simple_preprocess
import nltk
import os
nltk.download('stopwords')
from nltk.corpus import stopwords
# get the stopwords
stop_words = stopwords.words('english')
# add some relevant words to the list
stop_words.extend(['http','quot','new','notice','please', 'may','award','awards','application','foa','announcement','must','applications','proposal','applications','funding','provide','support','opportunity','grant','include','includes','eligible'])
# create a function to remove punctuation
def sent_to_words(documents):
    for document in documents:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(document), deacc=True))
# create a function to remove stopwords
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]
# get the descriptions as a list
grantDescriptions=list(grantsDF['Description'].values)
# remove the punctuation
grantDescriptions=sent_to_words(grantDescriptions)
# remove the stopwords
grantDescriptions=remove_stopwords(grantDescriptions)
print('Text cleaned')

# now for the stuff that takes a long time
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(grantDescriptions)
# Create Corpus
texts = grantDescriptions
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
print('Corpus creation complete')
# View
print(corpus[:1][0][:30])

from pprint import pprint
# number of topics
num_topics = 30
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dbullock\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Text cleaned
Corpus creation complete
[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1)]
[(6,
  '0.011*"program" + 0.008*"support" + 0.008*"research" + 0.007*"development" '
  '+ 0.006*"national" + 0.005*"public" + 0.005*"training" + 0.005*"funding" + '
  '0.004*"communities" + 0.004*"programs"'),
 (26,
  '0.016*"research" + 0.012*"program" + 0.009*"data" + 0.008*"may" + '
  '0.007*"applications" + 0.006*"information" + 0.006*"foa" + '
  '0.006*"development" + 0.005*"gov" + 0.005*"science"'),
 (1,
  '0.020*"application" + 0.013*"program" + 0.011*"information" + '
  '0.011*"federal" + 0.010*"funding" + 0.010*"gov" + 0.008*"grants" + '
  '0.008*"programs" + 0.008*"notice" + 0.007*"official"'),
 (7,
  '0.014*"national" + 0.014*"nps" + 0.013*"project" + 0.012*"park" + '
  '0.012*"pro

In [16]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
localPath=os.getcwd()
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join(os.path.split(localPath)[0],'results','ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath + '.pkl', 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath + '.pkl', 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, LDAvis_data_filepath +'.html')
LDAvis_prepared

C:\Users\dbullock\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19    -0.004501 -0.088177       1        1  10.781829
24    -0.071889  0.076847       2        1   5.993030
23    -0.036477 -0.001793       3        1   5.862921
0      0.204764  0.058979       4        1   5.726465
2     -0.004981 -0.077070       5        1   5.077050
13     0.023949 -0.064175       6        1   5.036099
6     -0.039482 -0.027028       7        1   5.029388
25    -0.011909  0.020137       8        1   4.742086
22    -0.044553  0.036237       9        1   4.675365
15    -0.021241 -0.038619      10        1   4.141111
1      0.003020  0.025089      11        1   3.832830
9     -0.022493  0.011714      12        1   3.400502
7     -0.062742  0.047393      13        1   3.333357
10     0.012088  0.007372      14        1   3.318543
21     0.013446  0.004564      15        1   2.781688
20     0.016945 -0.013093      16        1   2.661837
5      0.003584 -0.005579      17        1   2.647744
16     0.025146 -0.016301      18        1   2.448706
26     0.090752 -0.031114      19        1   2.265266
29     0.004717  0.013883      20        1   1.973700
11    -0.015690 -0.017355      21        1   1.836029
8     -0.018285 -0.028409      22        1   1.829307
12    -0.017327  0.036179      23        1   1.779691
17     0.003597  0.033243      24        1   1.766477
14    -0.007618 -0.024586      25        1   1.581723
27    -0.064590  0.020214      26        1   1.572111
18     0.078022  0.018221      27        1   1.147336
4     -0.028587 -0.002205      28        1   1.035171
28    -0.004800  0.026871      29        1   0.926000
3     -0.002864 -0.001437      30        1   0.796641, topic_info=              Term          Freq         Total Category  logprob  loglift
91        research  90086.000000  90086.000000  Default  30.0000  30.0000
46         program  92712.000000  92712.000000  Default  29.0000  29.0000
3643         roses  14442.000000  14442.000000  Default  28.0000  28.0000
365         health  42639.000000  42639.000000  Default  27.0000  27.0000
439    application  30671.000000  30671.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
106   applications    270.955323  36842.039494  Topic30  -5.5409  -0.0799
557        purpose    226.186388  21818.068438  Topic30  -5.7215   0.2634
523            may    239.184358  33021.403856  Topic30  -5.6656  -0.0952
929       projects    236.269145  31435.495377  Topic30  -5.6779  -0.0582
392         notice    226.525508  23312.213171  Topic30  -5.7200   0.1986

[2800 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
32911     14  0.815006    aacr
32911     21  0.045278    aacr
3848      11  0.740341  aadcrc
3848      29  0.092543  aadcrc
29824      8  0.253514    aahc
...      ...       ...     ...
16203     20  0.011762     zip
16203     27  0.003921     zip
16203     30  0.003921     zip
21630      9  0.869351    zrda
37941     16  0.835712     zur

[24304 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 25, 24, 1, 3, 14, 7, 26, 23, 16, 2, 10, 8, 11, 22, 21, 6, 17, 27, 30, 12, 9, 13, 18, 15, 28, 19, 5, 29, 4])